# DeepSTORM2D Add Ons
(1) Convert Picasso csv file to DeepSTORM2D csv format <br/>
*Input file:* Picasso localization file csv, header = frame, x [nm], y [nm], sigma [nm], intensity [photon], offset [photon], bkgstd [photon], uncertainty_xy [nm]. <br/>
*Output file:* DeepSTORM2D file csv, header = ,frame,x [nm],y [nm],Photon #,Sigma [nm]; frame idx starts at 1. <br/>

(2) Convert Picasso hdf5 file to DeepSTORM2D csv format <br/>
*Input file:* Picasso localization file hdf5, header = frame, x, y, photons, sx, sy, bg, lpx, lpy, ellipticity, net_gradient.  <br/>
*Output file:* DeepSTORM2D file csv, header = ,frame,x [nm],y [nm],Photon #,Sigma [nm]; frame idx starts at 1. <br/>

(3) Split tif movie & corresponding localization file at defined frame <br/>
*Input file:* Tif movie & csv file (Picasso or DeepSTORM2D format). <br/>
*Parameter split at frame:* Number of frames of the first file. <br/>
*Output file:* Two tif movies & csv files, split at defined frame. <br/>

(4) Merge multiple movies & localization files <br/>
*Input files:* Define multiple paths to movies and their localization files, adjust the number of files by deactivating "hide codecell" and adjusting the variable "n_measurements". Picasso or DeepSTORM2D csv formats are possible, but no mixing. <br/>
*Output file:* Tif movie with frames of movies stacked and localization file with continous frame numbering. <br/>

(5) Split into single frames <br/>
*Input file:* Tif movie & csv file  (Picasso or DeepSTORM2D format). <br/>
*Output files:* Single frames of tif movies with corresponding csv file saved in defined directory. <br/>

(6) Sum frames to create high density movie <br/>
*Input file:* Tif movie. <br/>
*Parameter camera offset:* Camera offset in px intensitiy. The n summed frames are corrected for multiple camera offsets by subtracting the value n-1 times. <br/>
*Parameter number of summing frames:* Define how many frames should be summed to create a high density frame. <br/>
*Output files:* Summed frames, each frame corresponds to a high density frame, containing summed information from n frames. \\  <br/>

In [1]:
from ImageBinner.widgets import widgetsAddOns, loadAddOns
from ImageBinner.tools import addOns
from ImageBinner.save import saveAddOns
import os
import warnings
warnings.filterwarnings("ignore")

## Convert localization file Picasso csv &rarr; DeepSTORM2D format

### Load

In [2]:
loc_conv_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
loc_conv_widget.file_button.on_click(loc_conv_widget.open_file)
loc_conv_widget.file_text_box.observe(loc_conv_widget.change_file_path_box)
display(loc_conv_widget.file_text_box, loc_conv_widget.file_button)

Text(value='', description='Path to localizations ', placeholder='Insert path', style=TextStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Convert & Save

In [3]:
def run(event):
    widget_run.create_clear_output()
    display(widget_run.run_button)
    if (loc_conv_widget.file_text_box.value):
        converter.convert(loc_conv_widget.file_text_box.value)
        display(converter.converted_file)
        saveAddOns.save_convert_file(loc_conv_widget.file_text_box.value, converter.converted_file)
    else:
        print("Please define a file to convert in the loading section.")
        
converter = addOns.Converter()
widget_run = widgetsAddOns.RunAnalysis("convert & save", "convert localization file")
display(widget_run.run_button)
widget_run.run_button.on_click(run)

Button(description='convert & save', style=ButtonStyle(), tooltip='convert localization file')

## Convert localization file Picasso hdf5 &rarr; DeepSTORM2D format

### Load

In [4]:
loc_conv_widget_h5 = widgetsAddOns.DefineFiles("localizations", title="Import *.hdf5 file", filetype=("hdf5 files", "*.hdf5"), idx="")
loc_conv_widget_h5.file_button.on_click(loc_conv_widget_h5.open_file)
loc_conv_widget_h5.file_text_box.observe(loc_conv_widget_h5.change_file_path_box)
display(loc_conv_widget_h5.file_text_box, loc_conv_widget_h5.file_button)

Text(value='', description='Path to localizations ', placeholder='Insert path', style=TextStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Convert & Save

In [5]:
def run(event):
    widget_run_h5.create_clear_output()
    display(widget_run_h5.run_button)
    if (loc_conv_widget_h5.file_text_box.value):
        converter_h5.convert(loc_conv_widget_h5.file_text_box.value)
        display(converter_h5.converted_file)
        saveAddOns.save_convert_file(loc_conv_widget_h5.file_text_box.value, converter_h5.converted_file)
    else:
        print("Please define a file to convert in the loading section.")
        
converter_h5 = addOns.ConverterH5()
widget_run_h5 = widgetsAddOns.RunAnalysis("convert & save", "convert localization file")
display(widget_run_h5.run_button)
widget_run_h5.run_button.on_click(run)

Button(description='convert & save', style=ButtonStyle(), tooltip='convert localization file')

### Save

## Split movie & localization file at defined frame

### Load

In [6]:
movie_split_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx="")
loc_split_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
movie_split_widget.file_button.on_click(movie_split_widget.open_file)
movie_split_widget.file_text_box.observe(movie_split_widget.change_file_path_box)
loc_split_widget.file_button.on_click(loc_split_widget.open_file)
loc_split_widget.file_text_box.observe(loc_split_widget.change_file_path_box)
display(movie_split_widget.file_text_box, movie_split_widget.file_button, loc_split_widget.file_text_box, loc_split_widget.file_button)

Text(value='', description='Path to movie ', placeholder='Insert path', style=TextStyle(description_width='ini…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations ', placeholder='Insert path', style=TextStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Parameters

In [7]:
widget_split_params = widgetsAddOns.ParameterBox("Split at frame", "Insert frame idx")
display(widget_split_params.value_box)

Text(value='', description='Split at frame', placeholder='Insert frame idx', style=TextStyle(description_width…

### Split

In [8]:
def run(event):
    widget_split_run.create_clear_output()
    display(widget_split_run.run_button)
    data_split.load_data(movie_split_widget.file_text_box.value, loc_split_widget.file_text_box.value)
    if len(data_split.loc_file["frame"].unique()) != data_split.movie.shape[0]:
        print("Defined movie and localization file differ in length, please make sure that they correspond.")
    else:
        split.split(data_split.movie, data_split.loc_file, int(widget_split_params.value_box.value))
data_split = loadAddOns.DataSplit()
split = addOns.Splitter()
widget_split_run = widgetsAddOns.RunAnalysis("split", "split data at frame")
display(widget_split_run.run_button)
widget_split_run.run_button.on_click(run)

Button(description='split', style=ButtonStyle(), tooltip='split data at frame')

### Save

In [9]:
def save_analysis(event):
    widget_split_save.create_clear_output()
    display(widget_split_save.save_button)
    saveAddOns.save_split_files(movie_split_widget.file_text_box.value, loc_split_widget.file_text_box.value, split.split_movies, split.split_locs, int(widget_split_params.value_box.value))

widget_split_save = widgetsAddOns.SaveResults()
display(widget_split_save.save_button)
widget_split_save.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Merge multiple movies & localization files

In [10]:
n_measurements = 2  # vary the number of measurements here!
movie_merge_widgets, loc_merge_widgets = [], []
for i in range(1, n_measurements+1):
    print("measurement", i)
    movie_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx=i)
    loc_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx=i)
    movie_widget.file_button.on_click(movie_widget.open_file)
    movie_widget.file_text_box.observe(movie_widget.change_file_path_box)
    loc_widget.file_button.on_click(loc_widget.open_file)
    loc_widget.file_text_box.observe(loc_widget.change_file_path_box)
    movie_merge_widgets.append(movie_widget)
    loc_merge_widgets.append(loc_widget)
    display(movie_widget.file_text_box, movie_widget.file_button, loc_widget.file_text_box, loc_widget.file_button)

measurement 1


Text(value='', description='Path to movie 1', placeholder='Insert path', style=TextStyle(description_width='in…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 1', placeholder='Insert path', style=TextStyle(description_w…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

measurement 2


Text(value='', description='Path to movie 2', placeholder='Insert path', style=TextStyle(description_width='in…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 2', placeholder='Insert path', style=TextStyle(description_w…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

In [11]:
def run(event):
    widget_merge_run.create_clear_output()
    display(widget_merge_run.run_button)
    movie_dirs = [widget.file_text_box.value for widget in movie_merge_widgets if widget.file_text_box.value]
    loc_dirs = [widget.file_text_box.value for widget in loc_merge_widgets if widget.file_text_box.value]
    data_merge.load_data(movie_dirs, loc_dirs)
    shapes = [movie.shape[1:] for movie in data_merge.movies]
    if all(x==shapes[0] for x in shapes):
        merge.merge(data_merge.movies, data_merge.loc_files)
    else:
        print("All movies must be of equal width and height.") 

data_merge = loadAddOns.DataMerge()
merge = addOns.Merger()
widget_merge_run = widgetsAddOns.RunAnalysis("merge", "merge files")
display(widget_merge_run.run_button)
widget_merge_run.run_button.on_click(run)

Button(description='merge', style=ButtonStyle(), tooltip='merge files')

In [12]:
widget_merge_save = widgetsAddOns.SaveResults()
display(widget_merge_save.box_filename, widget_merge_save.dir_box, widget_merge_save.dir_button)
widget_merge_save.dir_button.on_click(widget_merge_save.open_dir)
widget_merge_save.dir_box.observe(widget_merge_save.change_dir_box)

Text(value='', description='Filename', placeholder='name of folder', style=TextStyle(description_width='initia…

Text(value='', description='Directory', placeholder='Directory to save', style=TextStyle(description_width='in…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [13]:
display(widget_merge_save.save_button)
def save_merge(event):
    widget_merge_save.create_clear_output()
    display(widget_merge_save.save_button)   
    saveAddOns.save_merge_files(widget_merge_save.dir_box.value, widget_merge_save.box_filename.value, merge.merged_movies, merge.merged_locs)
widget_merge_save.save_button.on_click(save_merge)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Split into single frames

In [14]:
movie_single_split_widget = widgetsAddOns.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx="")
loc_single_split_widget = widgetsAddOns.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx="")
movie_single_split_widget.file_button.on_click(movie_single_split_widget.open_file)
movie_single_split_widget.file_text_box.observe(movie_single_split_widget.change_file_path_box)
loc_single_split_widget.file_button.on_click(loc_single_split_widget.open_file)
loc_single_split_widget.file_text_box.observe(loc_single_split_widget.change_file_path_box)
display(movie_single_split_widget.file_text_box, movie_single_split_widget.file_button, loc_single_split_widget.file_text_box, loc_single_split_widget.file_button)

Text(value='', description='Path to movie ', placeholder='Insert path', style=TextStyle(description_width='ini…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations ', placeholder='Insert path', style=TextStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

In [15]:
def run_single_split(event):
    widget_single_split_run.create_clear_output()
    display(widget_single_split_run.run_button)
    data_single_split.load_data(movie_single_split_widget.file_text_box.value, loc_single_split_widget.file_text_box.value)
    if len(data_single_split.loc_file["frame"].unique()) != data_single_split.movie.shape[0]:
        print("Defined movie and localization file differ in length, please make sure that they correspond.")
    else:
        single_split.split(data_single_split.movie, data_single_split.loc_file)
        
data_single_split = loadAddOns.DataSplit()
single_split = addOns.SingleSplitter()
widget_single_split_run = widgetsAddOns.RunAnalysis("split", "split data at every frame")
display(widget_single_split_run.run_button)
widget_single_split_run.run_button.on_click(run_single_split)

Button(description='split', style=ButtonStyle(), tooltip='split data at every frame')

In [16]:
widget_single_split_save = widgetsAddOns.SaveResults()
display(widget_single_split_save.dir_box, widget_single_split_save.dir_button)
widget_single_split_save.dir_button.on_click(widget_single_split_save.open_dir)
widget_single_split_save.dir_box.observe(widget_single_split_save.change_dir_box)

Text(value='', description='Directory', placeholder='Directory to save', style=TextStyle(description_width='in…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [17]:
display(widget_single_split_save.save_button)
def save_single_split(event):
    widget_single_split_save.create_clear_output()
    display(widget_single_split_save.save_button)   
    saveAddOns.save_single_split_files(widget_single_split_save.dir_box.value, os.path.splitext(os.path.split(loc_single_split_widget.file_text_box.value)[1])[0], single_split.single_frame_movies, single_split.single_frame_locs)
widget_single_split_save.save_button.on_click(save_single_split)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Sum frames to create high density movie

### Load

In [18]:
widget_load_sum = widgetsAddOns.DefineFiles("tif movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx="")
widget_load_sum.file_button.on_click(widget_load_sum.open_file)
widget_load_sum.file_text_box.observe(widget_load_sum.change_file_path_box)
display(widget_load_sum.file_text_box, widget_load_sum.file_button)

Text(value='', description='Path to tif movie ', placeholder='Insert path', style=TextStyle(description_width=…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Parameter
*Camera offset:* Camera offset in px intensitiy. The n summed frames are corrected for multiple camera offsets by subtracting the value n-1 times.
*Number of summing frames:* Define how many frames should be summed to create a high density frame. 

In [19]:
widget_sum_params_camera = widgetsAddOns.ParameterBox("Camera offset [px intensity]", "Insert value")
widget_sum_params_sum = widgetsAddOns.ParameterBox("Number of frames to sum together", "Insert value")
display(widget_sum_params_camera.value_box)
display(widget_sum_params_sum.value_box)

Text(value='', description='Camera offset [px intensity]', placeholder='Insert value', style=TextStyle(descrip…

Text(value='', description='Number of frames to sum together', placeholder='Insert value', style=TextStyle(des…

### Save

In [20]:
def run(event):
    widget_sum_run.create_clear_output()
    display(widget_sum_run.run_button)
    if (widget_load_sum.file_text_box.value):
        movie = loader_sum.load_movie(widget_load_sum.file_text_box.value)
        summer.sum_movie(loader_sum.movie, int(widget_sum_params_sum.value_box.value), int(widget_sum_params_camera.value_box.value))
        save_path = os.path.splitext(widget_load_sum.file_text_box.value)[0] + "_n_sum" + widget_sum_params_sum.value_box.value + "_offset" + widget_sum_params_camera.value_box.value + ".tif"
        saveAddOns.save_sum_frames(save_path, summer.summed_frames)
    else:
        print("Please define a file in the loading section.")

loader_sum = loadAddOns.SumFrames()
summer = addOns.SumFrames()
widget_sum_run = widgetsAddOns.RunAnalysis("sum & save", "sum frames to one")
display(widget_sum_run.run_button)
widget_sum_run.run_button.on_click(run)

Button(description='sum & save', style=ButtonStyle(), tooltip='sum frames to one')